In [31]:
import pandas as pd
store = pd.read_csv('store.csv')

In [25]:
# Check for missing values
print(store.isnull().sum())

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64


In [26]:
print(store.dtypes)

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object


In [32]:
train = pd.read_csv('train.csv')

<ipython-input-32-f1a1f2ce149f>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [33]:
train['StateHoliday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [34]:
#deal with 3 missing value in Competition Distance
store_copy = store.copy()
store = store.dropna(subset=['CompetitionDistance'])

corr = store['CompetitionDistance'].corr(train['Sales'])
corr
#Since the correlation between CompetitionDistance and Sales is low,
#indicating that CompetitionDistance is less likely to significantly impact sales, I decided to drop this feature.

0.004085248059166643

In [35]:
#store = store.dropna(subset=['CompetitionDistance'])
#dropping rows with missing values in CompetitionDistance is really the best way? cuase deleting these rows would remove other data for three stores
#or could drop the CompetitionDistance column entirely or exclude it from the feature selection when predicting.
store['CompetitionDistance'] = store['CompetitionDistance'].fillna(store['CompetitionDistance'].mean())

In [36]:
#deal with 354 missing value in CompetitionOpenSinceMonth & CompetitionOpenSinceYear
#Since the number of missing values is large, deleting all of them would be inappropriate.
#Additionally, since the data is not sequential, forward/backward fill is not suitable.
#Therefore, I will use the median to impute the missing values.

store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear'].median())
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(store['CompetitionOpenSinceMonth'].median())

In [38]:
#deal with 544 missing value in Promo2SinceWeek,Promo2SinceYear,PromoInterval
#Since the missing values occur because Promo2 is 0, indicating that the store is not participating in the promotion
#I will use 0 to fill the missing values.
store['Promo2SinceWeek'] = store['Promo2SinceWeek'].fillna(0)
store['Promo2SinceYear'] = store['Promo2SinceYear'].fillna(0)
store['PromoInterval'] = store['PromoInterval'].fillna('None')
# last filling is None rather than 0 is for the conveniance of later categoraical encoding. because when I use One K hot, all 0 could mean some month rather than missing value.
#in order to classify the situation, I will use 'None' to impute

In [39]:
print(store.isnull().sum())

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64


In [41]:
#encode PromoInterval
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# 函数：将PromoInterval字段的字符串分割，并返回一个包含所有月份的序列，缺失的月份用0填充
def encode_promo(promo_str):
    # 如果是“None”，则返回全部为0的序列
    if promo_str == 'None':
        return pd.Series([0] * len(months), index=months)
    else:
        # 否则，将字符串分割成列表，并创建一个序列，存在于列表中的月份标记为1，不存在的标记为0
        promo_list = promo_str.split(',')
        return pd.Series([1 if month in promo_list else 0 for month in months], index=months)

# 应用函数，并创建新的DataFrame
promo_encoded = store['PromoInterval'].apply(encode_promo).apply(pd.Series)

# 将新的编码DataFrame与原始DataFrame合并
store = pd.concat([store, promo_encoded], axis=1)

# 删除原始的PromoInterval列
store.drop('PromoInterval', axis=1, inplace=True)

In [43]:
print(store.isnull().sum())

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
Jan                          0
Feb                          0
Mar                          0
Apr                          0
May                          0
Jun                          0
Jul                          0
Aug                          0
Sep                          0
Oct                          0
Nov                          0
Dec                          0
dtype: int64


In [56]:
train = train.merge(store, left_on='Store', right_index=True, how='left')
train = train.drop(columns=['Store_x'])

In [57]:
train.head(1116)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Store_y,...,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1,5,31/07/2015,5263,555,1,1,0,1,2.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2,5,31/07/2015,6064,625,1,1,0,1,3.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,5,31/07/2015,8314,821,1,1,0,1,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,5,31/07/2015,13995,1498,1,1,0,1,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5,31/07/2015,4822,559,1,1,0,1,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,1112,5,31/07/2015,9626,767,1,1,0,1,1113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1112,1113,5,31/07/2015,7289,720,1,1,0,1,1114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1113,1114,5,31/07/2015,27508,3745,1,1,0,1,1115.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1114,1115,5,31/07/2015,8680,538,1,1,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
test = pd.read_csv('test.csv')

In [63]:
test = test.merge(store, left_on='Store', right_index=True, how='left')
test = test.drop(columns=['Store_x'])

In [64]:
test.head(1115)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Store_y,...,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1,4,17/09/2015,NaN,NaN,1.0,1,0,0,2.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,3,4,17/09/2015,NaN,NaN,1.0,1,0,0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,4,17/09/2015,NaN,NaN,1.0,1,0,0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,4,17/09/2015,NaN,NaN,1.0,1,0,0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,4,17/09/2015,NaN,NaN,1.0,1,0,0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,333,3,16/09/2015,NaN,NaN,1.0,1,0,0,334.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1111,334,3,16/09/2015,NaN,NaN,1.0,1,0,0,335.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1112,335,3,16/09/2015,NaN,NaN,1.0,1,0,0,336.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1113,336,3,16/09/2015,NaN,NaN,1.0,1,0,0,337.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
